In [ ]:
# Optional: single-threaded BLAS to avoid oversubscription on laptops
import os
os.environ['OMP_NUM_THREADS'] = '1'
os.environ['OPENBLAS_NUM_THREADS'] = '1'


# End-to-end MCMC

This notebook demonstrates how to:
1) Select the active data (Cosmic Chronometers, Megamasers, Supernovae, DESI BAO, Planck distance priors) **implicitly** from your current redshift bin \([a,b]\) and toggles using `likelihood.build_total_chi2`.
2) Automatically construct the parameter vector in the correct order (`param_names`) for minimization and MCMC implementation.
3) Run a local minimization to find good starting points.
4) Sample the posterior with `emcee` and visualize results with `corner`.

> Requirements: the `likelihood.py`, `cosmology.py`, `bin_range.py` and the data files must be available/importable in the same environment where you open this notebook.


In [ ]:
import numpy as np
import scipy.optimize as opt
import emcee, corner
#from multiprocessing import Pool                                # Use this for Linux 
from pathos.multiprocessing import ProcessingPool as Pool        # Use this for MacOS
import likelihood
import bin_range


print("Active z-bin:", bin_range.zmin, "to", bin_range.zmax)
# Check whether Planck distance priors are active

planck_dp_status = getattr(bin_range, 'Planck_distance_priors', False)
if planck_dp_status:
    print("Planck Distance Priors are ACTIVE (chi2_PlanckDP will be included).")
else:
    print("Planck Distance Priors are DISABLED (chi2_PlanckDP will return 0).")


## 1) Choose the SNe sample in "Base+SNe"

Pick one among `"PantheonPlus"`, `"Union3"`, or `"DES"`.  
`build_total_chi2` returns:
- `chi2_fn(theta)`: a callable expecting exactly the parameters listed in
- `param_names`: the ordered list of parameter names for the active configuration.

In [ ]:

# Choose which Base+SNe sample to use here:
which_sne = "PantheonPlus"  # choose between "PantheonPlus", "Union3" or "DES" for the corresponding Base+SNe likelihood

chi2_fn, param_names = likelihood.build_total_chi2(which_sne=which_sne)
print("Parameter order for this configuration:")
print(param_names)
ndim = len(param_names)

## 2) Priors and log-posterior

Define broad, sensible priors per-parameter. Masers velocities use a top-hat prior around measured values; other parameters have wide top-hat ranges that you can tighten as needed.


In [ ]:

# Measured maser velocities for a top-hat prior (center ± width)
v0 = dict(v1=3319.9, v2=10192.6, v3=7801.5, v4=8525.7, v5=7172.2, v6=679.3)
vwidth = 1000.0  # km/s

# Broad uniform bounds for cosmological/other params
bounds = {
    "Om0": (0.0, 1.0),
    "H0":  (10.0, 150.0),
    "w0":  (-2.0, -1.0/3.0),
    "wa":  (-5.0, 5.0),
    "M":   (-25.0, +5.0), # Absolute M for Pantheon+; for Union3/DES (ΔM) this is intentionally wide. Remember that DES and Union3 fit the offset ΔM
    "rd":  (100, 200.0),
    "Omega_bh2": (0.0, 0.05),
}

def lnprior(theta):
    # Top-hat priors from dictionaries above
    for name, value in zip(param_names, theta):
        if name in v0:
            if not (v0[name] - vwidth < value < v0[name] + vwidth):
                return -np.inf
        elif name in bounds:
            lo, hi = bounds[name]
            if not (lo < value < hi):
                return -np.inf
        else:
            # If a parameter appears with no bound, accept it (rare)
            pass
    return 0.0

def lnposterior(theta):
    lp = lnprior(theta)
    if not np.isfinite(lp):
        return -np.inf
    chi2 = chi2_fn(theta)
    val = -0.5 * chi2 + lp
    if np.isnan(val):
        return -np.inf
    return val

## 3) initial walkers

This block defines the fixed seed parameters and automatically builds the initial walker positions for the MCMC.

In [ ]:
# --- Seeds: fixed reference values (edit as desired) ---
# Maser galaxy velocities:
v1 = 3319.9
v2 = 10192.
v3 = 7801.5
v4 = 8525.7
v5 = 7172.2
v6 = 679.3

Omega_m    = 0.3       # Matter density parameter 
H0         = 70        # H0 [km/s/Mpc]
w0         = -1        # CPL present-day DE EoS
wa         =  0        # CPL evolution
M_pantheon = -19.3     # Absolute magnitude for Pantheon+
M_Union3   = 0         # Union3 fits ΔM
M_DES      = 0         # DES fits ΔM
rd         = 144       # Sound horizon [Mpc]
Omega_bh2  = 0.02      # Ω_b h^2

def choose_M(which):
    if which == "PantheonPlus":
        return M_pantheon
    elif which == "Union3":
        return M_Union3
    elif which == "DES":
        return M_DES
    raise ValueError("Unknown SNe selector")

# Assemble the seed vector exactly in the order required by `param_names`
seed_dict = {
    "v1": v1, "v2": v2, "v3": v3, "v4": v4, "v5": v5, "v6": v6,
    "Om0": Omega_m, "H0": H0, "w0": w0, "wa": wa,
    "M": choose_M(which_sne),
    "rd": rd,
    "Omega_bh2": Omega_bh2,
}

def assemble_vector_from_names(names, values_dict):
    return np.array([values_dict[n] for n in names], dtype=float)

theta0 = assemble_vector_from_names(param_names, seed_dict)

# --- Walker cloud around theta0 (no minimization) ---
ndim = len(param_names)
nwalkers = max(2*ndim + 2, 30)  # common heuristic

# Per-parameter proposal scales (fallback to a small default if not listed)
default_scales = {
    "v1":10, "v2":10, "v3":10, "v4":10, "v5":10, "v6":10,
    "Om0":0.05, "H0":10.0, "w0":0.1, "wa":0.1,
    "M":0.1, "rd":10.0, "Omega_bh2":0.001,
}
scales = np.array([ default_scales.get(n, 0.01) for n in param_names ], dtype=float)

rng = np.random.default_rng(1234)
p0 = theta0 + rng.normal(scale=scales, size=(nwalkers, ndim))


## 4) Run the MCMC with `emcee`

This block uses a process pool if available. Adjust `nsteps` to your needs.


In [ ]:
nsteps = 15000  # adjust as needed

with Pool() as pool:
    sampler = emcee.EnsembleSampler(nwalkers, ndim, lnposterior, pool=pool)
    sampler.run_mcmc(p0, nsteps, progress=True)

samples = sampler.get_chain()
#np.save("raw_chain.npy", samples) # Save raw chain if desired


## 7) Convergence diagnostics and thinning


In [ ]:
try:
    tau = sampler.get_autocorr_time()
    discard = int(nsteps/3)
    thin = max(1, int(np.nanmin(tau)/2))
    print("Autocorr time:", tau)
except Exception as e:
    print("Autocorr-time estimation failed:", e)
    # Very conservative fallback
    discard = int(nsteps/10)
    thin = max(1, int((nsteps - discard)//100))

flat_samples = sampler.get_chain(discard=discard, thin=thin, flat=True)
# np.save("flat_samples.npy", flat_samples)


## 8) Corner plot

By default, we drop the six maser nuisance velocities from the plot (if present) and show the rest.


In [ ]:

# Select parameters to plot (skip maser v1..v6 if present)
plot_mask = [not (name.startswith('v') and len(name)==2 and name[1].isdigit()) for name in param_names]
plot_cols = np.where(plot_mask)[0]
labels = [param_names[i] for i in plot_cols]

fig = corner.corner(flat_samples[:, plot_cols], labels=labels,
                    plot_contours=True, plot_datapoints=False,
                    show_titles=True, smooth=True)


## Notes and tips

- The **active parameters** and their **order** are controlled entirely by `likelihood.build_total_chi2`. If you change the redshift bin in `bin_range.py` or toggle Planck distance priors, simply re-run the cell that calls `build_total_chi2` and the rest of the notebook will adapt automatically.
- If you switch the SN sample (`which_sne`), the seed for `M` is chosen accordingly (`M_pantheon` vs. offsets for Union3/DES).
- Priors here are broad and simple; tailor them to your scientific goals.
- To use a different optimizer or add bounds, consider `scipy.optimize.differential_evolution` or `opt.minimize` with `method="Powell"` / `method="L-BFGS-B"` (with bounds).
- For speed, you can reduce `nsteps` during development and increase it for your final runs.
